<a href="https://colab.research.google.com/github/nurulc/ATM/blob/master/tf_tutor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [120]:
import numpy as np
#from google.colab import drive
import ipywidgets as widgets
import altair as alt

#import os

In [3]:
![[ -f housing-ma.csv ]] || echo no

no


# Fetch MA Housing data from github

In [5]:

#drive.mount('content/gdrive')
# !head drive/My\ Drive/housing-ma.csv
#!head "/content/drive/My Drive/Colab Notebooks/Copy of tf-tutor.ipynb"

![[ -f housing-ma.csv ]] || wget https://raw.githubusercontent.com/nurulc/ATM/master/housing-ma.csv 

!head housing-ma.csv



--2020-07-30 20:04:03--  https://raw.githubusercontent.com/nurulc/ATM/master/housing-ma.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41359 (40K) [text/plain]
Saving to: ‘housing-ma.csv’

housing-ma.csv      100%[===================>]  40.39K  --.-KB/s    in 0.02s   

2020-07-30 20:04:03 (2.61 MB/s) - ‘housing-ma.csv’ saved [41359/41359]

CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0.00632,18.00,2.310,0,0.5380,6.5750,65.20,4.0900,1,296.0,15.30,396.90,4.98,24.00
0.02731,0.00,7.070,0,0.4690,6.4210,78.90,4.9671,2,242.0,17.80,396.90,9.14,21.60
0.02729,0.00,7.070,0,0.4690,7.1850,61.10,4.9671,2,242.0,17.80,392.83,4.03,34.70
0.03237,0.00,2.180,0,0.4580,6.9980,45.80,6.0622,3,222.0,18.70,394.63,2.94,33.40
0.06905,0.00,2.180,0,0.4580,7.1470,54.20,6

In [4]:
!ls


drive  gdrive  housing-ma.csv  sample_data


# Read MA housing data into a pandas table

In [6]:
import pandas as pd
import io
data_pd = pd.read_csv(io.FileIO('housing-ma.csv'))
data_pd


,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


# Chart some of the data

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV']

In [119]:
['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']

alt.Chart(data_pd).mark_point().encode(
    x='AGE', y="NOX"
).interactive()


alt.Chart(...)

In [87]:
#data_pd.drop(['MEDV'])
data_pd.describe()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
count,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000,506.000000
mean,3.613524,11.363636,11.136779,0.069170,0.554695,6.284634,68.574901,3.795043,9.549407,408.237154,18.455534,356.674032,12.653063,22.532806
std,8.601545,23.322453,6.860353,0.253994,0.115878,0.702617,28.148861,2.105710,8.707259,168.537116,2.164946,91.294864,7.141062,9.197104
min,0.006320,0.000000,0.460000,0.000000,0.385000,3.561000,2.900000,1.129600,1.000000,187.000000,12.600000,0.320000,1.730000,5.000000
25%,0.082045,0.000000,5.190000,0.000000,0.449000,5.885500,45.025000,2.100175,4.000000,279.000000,17.400000,375.377500,6.950000,17.025000
50%,0.256510,0.000000,9.690000,0.000000,0.538000,6.208500,77.500000,3.207450,5.000000,330.000000,19.050000,391.440000,11.360000,21.200000
75%,3.677082,12.500000,18.100000,0.000000,0.624000,6.623500,94.075000,5.188425,24.000000,666.000000,20.200000,396.225000,16.955000,25.000000
max,88.976200,100.000000,27.740000,1.000000,0.871000,8.780000,100.000000,12.126500,24.000000,711.000000,22.000000,396.900000,37.970000,50.000000


In [6]:
dtype=np.double
def col_sort(t,col=0,descending=False):
    return t[t[:, col].sort(descending=descending)[1]]



# Normalize the data (function)

For each column of X(data)  subtract the mean of the column and scale by the range $$(highest - lowest)$$

This normalization maked the algoririthm more stable

> _Note:_ <span color="red">Do not mormalize the price column (MEDV)</span>

In [7]:
def norm(X):
    R=(np.amax(X,0)-np.amin(X,0)).reshape(-1,1)
    R[X.shape[1]-1][0] = 1.0
    M=np.mean(X,0).reshape(-1,1)
    M[X.shape[1]-1][0] = 0.0
    Mv = M.dot(np.ones((1,X.shape[0]),dtype=dtype)).transpose()
    Sc = R.dot(np.ones((1,X.shape[0]),dtype=dtype)).transpose()
    return [(X-Mv)/Sc,R,M]

# Setup MA housing data for regression

1. Convert pandas data to numpy
2. Place in X all the columns except the price
3. Add a '1's column to X _(this is to incorporate the intercept)_
4. Put into y the house prices where $y_i$ price corresponding to row $X_i$ 
5. Initialize $\phi$ with '1's


In [80]:


# !head sample_data/housing-ma.csv
ones = np.ones
cat = np.concatenate
def setup(data_pd,drop=[]):
    data_pd = data_pd.drop(drop,axis=1)
    Z = data_pd.to_numpy(dtype=np.double,copy=True)
    m = Z.shape[0] # m = number of rows of data
    n = Z.shape[1]-1 # n = number of columns excluding the house price
    X = Z[:,0:n] # remove the house price (last column)

    Ones = ones(m,dtype=np.double).reshape(-1,1)
    X = cat((X,Ones),1) # add a 1 to each row (to compute the intercept)
    y = Z[:,n].reshape(-1,1) # get the house price
    phi = np.ones(n+1,dtype=np.double).reshape(-1,1) # initial value of phi
    return [X, y, phi]



# Numpy Multivariate regression

This method uses gradient descent to iteratively mimimize the cost function (MSE - mean standard error).

## Iterative computation of parameters _phi_ (multi variable linear regression)

$phi (\phi) $ are the regression parematers

*  find $\phi$ that minimizes cost
  *  *where:*    $Cost^2 = MSE = \frac{(X\phi -y)^T(X\phi -y)}{2m}$
   * $\phi$ is a minimum when $\frac{\partial{(Cost^2)}}{\partial\phi} = 0$
   * i.e. $\frac{\partial{(Cost^2)}}{\partial\phi} = \frac{X^T(X\phi - y)}{m} = 0$
   * by gradient descent we can compute successive values of $\phi_j$ 
       * $  \phi_j = \phi_{j-1} - \alpha \frac{\partial{(Cost^2)}}{\partial\phi}$
       * Resulting in the following:
          * $  \phi_j = \phi_{j-1} - \alpha X^T(X\phi_{j-1} - y)$
          * $\alpha$ is the learning rate is a small scalar such as 0.05

#### Where  
*X* = input data of [$m \times n$] matrix 

*  m = number of data rows, 
*  n = number of independent attributes of each data row

*y* = output (house price), vector of length $m$

_phi_ ($\phi$) = regression coefficients
and

_Where_: X$\phi$ (matrix multiply) = prediced house price

$Predicted_i = \sum_{j=1}^{n} X_{ij} * \phi_j$

In [9]:
import math
tr = np.transpose

def cost(X,phi,Y):
    m = X.shape[0];
    return math.sqrt((1/(2*m))*(tr(X@phi - Y)@(X@phi - Y)).item())

# Gradient decent for finding $\phi$ to minimise cost

In [10]:


def npSolve(X,y,iter=2,alpha=0.05):
    W, Range,Mean = norm(X) # normalize the data

    phi = ones((X.shape[1],1),dtype=np.double)
    cost_list = []
    Y = y
    m = X.shape[0]
    
    for i in range(iter):
        delta = alpha*(1/m)*(W.T)@(W@phi - Y) 
        phi = phi - delta
        cost_val = cost(W,phi,Y)
        cost_list.append(cost_val)
        
    intercept = np.zeros_like(Mean)
    intercept[-1][0] = (phi*Mean/Range).sum().item()
    phi = phi/Range - intercept
    return [phi,cost_list]    

# Iterative computation of parameters _phi_ (multi variable linear regression)

#### Where  
*X* = input data of [$m \times n$] matrix 

*  m = number of data rows, 
*  n = number of independent attributes of each data row

*y* = output (house price), vector of length $m$

_phi_ ($\phi$) = regression coefficients
and

_Where_: X$\phi$ (matrix multiply) = prediced house price

$Predicted_i = \sum_{j=1}^{n} X_{ij} * \phi_j$


        

In [92]:
#%%time
X, y, phi = setup(data_pd, drop=[])   

#print("input\n", X)
#print("house price (first 10)\n",y[0:10])

phi,cost_list = npSolve(X,y,500,0.5)

print(phi, cost(X,phi,y))

pl = pd.DataFrame({"iteration": range(len(cost_list)),
                  "cost":cost_list})

alt.Chart(pl).mark_point().encode(
    x='iteration', y="cost"
).interactive()

[[-7.19119735e-02]
 [ 3.80905066e-02]
 [ 4.24813183e-03]
 [ 2.79127809e+00]
 [-1.51363271e+01]
 [ 3.87847148e+00]
 [ 2.33877353e-03]
 [-1.28425309e+00]
 [ 2.54373661e-01]
 [-1.03697237e-02]
 [-9.41854971e-01]
 [ 9.67083950e-03]
 [-5.39282499e-01]
 [ 3.34149801e+01]] 3.31686406768445


alt.Chart(...)

In [61]:
X1,Range,Mean = norm(X)
Mean

array([[3.61352356e+00],
       [1.13636364e+01],
       [1.11367787e+01],
       [6.91699605e-02],
       [5.54695059e-01],
       [6.28463439e+00],
       [6.85749012e+01],
       [3.79504269e+00],
       [9.54940711e+00],
       [4.08237154e+02],
       [1.84555336e+01],
       [3.56674032e+02],
       [1.26530632e+01],
       [0.00000000e+00]])

# Perform Multivariate regression (directly - no iteration)

We previously saw that for minimum $\phi$
* We have the following
* $\frac{X^T(X\phi - y)}{m} = 0$
  * multiplying bost sides by $m$
* $X^T(X\phi - y) = 0$
  * Multiplying by $X^T$ - _distributive property_
* $X^T(X\phi) -  X^Ty = 0$
  * Rearanging
* $(X^TX)\phi =  X^Ty$
----
   * Multiply both sides by the inverse of $X^TX$

* $(X^TX)^{-1} (X^TX)\phi = (X^TX)^{-1} X^Ty$
   * since  ${(X^TX)^{-1} (X^TX)} = I (identity)$
* we get the following: $I \phi = (X^TX)^{-1} X^Ty$
----
 **Therefore we get the very simple formula for $\phi$**
   * *$\phi = (X^TX)^{-1} X^Ty$*
----
&nbsp;

Note the following rules for matrix multiply
* $A(BC) = (AB)C$
  * and
* $A(B+C) = AB + BC$ - _distributive property_



In [89]:
%%time

inv = np.linalg.inv # invert a matrix

# See how simple the formulation can be using only linear algebra
def solve(X,y):
  return inv(X.T@X) @ (X.T@y)

phi2 = solve(X,y)  
print("cost=", cost(X,phi2,y),"diff = ", cost(X,phi,y)-cost(X,phi2,y)) # compare the cost of direct method and iterative method
#[phi,phi2, cost(X,phi2,y)-cost(X,phi,y), np.abs((phi-phi2)/phi*100)]

cost= 3.341225742387936 diff =  0.008488686388040456
CPU times: user 2.07 ms, sys: 2.93 ms, total: 5 ms
Wall time: 5.49 ms


# Draw chart of actual price against prediced price


In [91]:
results = pd.DataFrame({"predicted": (X@phi2).reshape(-1),
                        "actual": y.reshape(-1)})

alt.Chart(results).mark_point().encode(
    x='actual', y="predicted",
    tooltip=['actual', 'predicted']
).interactive()

alt.Chart(...)

In [55]:
R, actual, phi = setup(results)   

#print("input\n", X)
#print("house price (first 10)\n",y[0:10])

phi_r = solve(R,actual)
print(phi_r, cost(R,phi_r,actual))
results = pd.DataFrame({"predicted": (R@phi_r).reshape(-1),
                        "actual": actual.reshape(-1)})

alt.Chart(results).mark_point().encode(
    x='actual', y="predicted",
    tooltip=['actual', 'predicted']
).interactive()

[[ 1.00617842]
 [-0.13921714]] 3.3166866717115715


alt.Chart(...)

In [107]:
# from vega_datasets import data

# source = data.cars()

# alt.Chart(source).mark_circle(size=60).encode(
#     x='Horsepower',
#     y='Miles_per_Gallon',
#     color='Origin',
#     tooltip=['Name', 'Origin', 'Horsepower', 'Miles_per_Gallon']
# ).interactive()

In [106]:
# np.random.seed(1)

# source = pd.DataFrame({
#     'x': np.arange(100),
#     'A': np.random.randn(100).cumsum(),
#     'B': np.random.randn(100).cumsum(),
#     'C': np.random.randn(100).cumsum(),
# })

# base = alt.Chart(source).mark_circle(opacity=0.5).transform_fold(
#     fold=['A', 'B', 'C'],
#     as_=['category', 'y']
# ).encode(
#     alt.X('x:Q'),
#     alt.Y('y:Q'),
#     alt.Color('category:N')
# )

# base + base.transform_loess('x', 'y', groupby=['category']).mark_line(size=4)


In [9]:
data_pd.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX',
       'PTRATIO', 'B', 'LSTAT', 'MEDV'],
      dtype='object')

In [105]:


from sklearn import preprocessing
def showPlot(key=""):
  c = list(data_pd.columns);
  c.remove(key)
  c.remove('MEDV')
  df = data_pd.drop(c,axis=1).sort_values(['MEDV'])
  ln = len(df.columns)
  cols = list(df.columns)
  x = df.values[:,0:ln-1] #returns a numpy array
  min_max_scaler = preprocessing.MinMaxScaler()
  x_scaled = min_max_scaler.fit_transform(x)
  # print(np.concatenate((x_scaled,df.values[:,ln-1].reshape(-1,1)), axis=1))
  df = pd.DataFrame(np.concatenate((x_scaled,df.values[:,ln-1].reshape(-1,1)),axis=1))
  # print("DF",df)
  newCols = dict(list(zip( list(range(0,ln)), cols)))
  df= df.rename(columns=newCols)
  # print(df)
  cols.remove('MEDV') 
  base = alt.Chart(df).mark_circle(opacity=0.5).transform_fold(
      fold=cols ,              #['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT'],
      as_=['category', 'y']
  ).encode(
      alt.X('MEDV:Q'),
      alt.Y('y:Q'),
      alt.Color('category:N')
  )
  
  return base + base.transform_loess('x', 'y', groupby=['category']).mark_line(size=4)
  return df

#####
print(list(data_pd.columns))
showPlot('PTRATIO')


['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV']


alt.LayerChart(...)

In [94]:
dict(list(zip( list(range(1,14)), list(data_pd.columns))))
# list(zip( list(range(0,ln-1)), cols))
data_pd

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222.0,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273.0,21.0,391.99,9.67,22.4
502,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273.0,21.0,396.90,9.08,20.6
503,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273.0,21.0,396.90,5.64,23.9
504,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273.0,21.0,393.45,6.48,22.0


In [68]:
list(data_pd.columns).remove('MEDV')